In [ ]:
#!pip install qdrant-client[fastembed]  # Includes local/embedded mode

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with mean pooling.


In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.models import PointStruct, VectorParams, Distance

# -------------------------------------------------------------------
# 1. Connect to Docker Qdrant (source)
# -------------------------------------------------------------------
docker_client = QdrantClient(host="localhost", port=6333)  # Your Docker Qdrant

# -------------------------------------------------------------------
# 2. Connect to Embedded Qdrant (target)
# -------------------------------------------------------------------
local_client = QdrantClient(path="./qdrant_data")  # Embedded mode

# -------------------------------------------------------------------
# 3. Recreate Collection Configuration in Embedded Qdrant
# -------------------------------------------------------------------
local_client.create_collection(
    collection_name="Auditron_legal_chunks",
    vectors_config=models.VectorParams(size=1024, distance=Distance.COSINE)
)

# -------------------------------------------------------------------
# 4. Transfer All Data in Batches
# -------------------------------------------------------------------
has_more = True
next_offset = None

while has_more:
    # Get batch from Docker Qdrant
    records, next_offset = docker_client.scroll(
        collection_name="Auditron_legal_chunks",
        limit=100,  # Adjust batch size based on memory
        offset=next_offset,
        with_payload=True,
        with_vectors=True
    )
    
    # Insert into Embedded Qdrant
    if records:
        local_client.upsert(
            collection_name="Auditron_legal_chunks",
            points=records
        )
    
    has_more = next_offset is not None

print("Migration complete!")

Migration complete!


In [6]:
from langchain_qdrant import QdrantVectorStore
vector_store = QdrantVectorStore(
    client=local_client,
    collection_name="Auditron_legal_chunks",  # Your collection name
    content_payload_key="text", 
    embedding=embeddings,
)
#    search_params=SearchParams(hnsw_ef=128)  # Your search params
# 4. Create the retriever from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 5. Use the retriever
query = "Quel est le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie depuis l'adoption de la loi N° 2020-46 du 23 décembre 2020?"
documents = retriever.invoke(query)

# The retrieved documents will include content and metadata
for doc in documents:
    print(doc.page_content)  # Access the content
    print(doc.metadata)      # Access the metadata

Pour la retenue à la source au titre des revenus de capitaux mobiliers L’article 17 de la loi n°2020-46 du 23 décembre 2020 portant loi de finances pour l'année 2021 a révisé le régime fiscal des revenus de capitaux mobiliers, et ce, par la généralisation de l'application de la retenue à la source libératoire au taux de 20% à tous les revenus de capitaux mobiliers. Ladite retenue à la source est définitive et non susceptible de déduction ou de restitution, et ce, nonobstant le régime fiscal et le résultat q
{'doc_type': 'notes_communes', 'doc_year': 2022, 'doc_title': 'note_commune_01_2022', 'doc_language': 'fr', 'token_count': 247, 'source_url': 'https://jibaya.tn/wp-content/uploads/2024/02/Note-Commune-N%C2%B001-1.pdf', '_id': '2f29e3bf-69cb-9211-4d32-ca2a1d5e5554', '_collection_name': 'Auditron_legal_chunks'}
Pour la retenue à la source Suite à l'augmentation du taux de l'impôt sur les sociétés de 15% à 20%, les taux de retenue à la source dus au titre des opérations de cession par 